<a href="https://colab.research.google.com/github/Wildemberg-Projects/Plants-Diseases/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Instalação de Dependências
# Para utilização do Colab, modifique o '%' por '!'
# As abaixo comentadas são somente para instalar em ambiente local
# %pip install -q -U tensorflow
# %pip install -q -U keras
# %pip install -q -U numpy
# %pip install -q -U pandas
# %pip install -q -U Jinja2
# Uso da GPU precisa ser habilitado pelo comando a seguir
# %python -m pip install tensorflow[and-cuda]
%pip install -q -U tensorflow-addons
%pip install -q -U tensorflow-datasets
%pip install -q -U keras-utils
%pip install -q -U matplotlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import tensorflow_datasets as tfds
import pandas as pd
from numpy import mean
import tensorflow as tf
import keras

In [8]:
# Coletando dados
tf.keras.backend.clear_session()
(ds_train), info = tfds.load('plant_village', split='train', with_info=True)

# a estrutura de uma imagem (244,244,3), representa a largura e altura em pixels,
# e o 3 representa a quantidade de canais de cores pq é RGB

2024-04-20 15:49:49.253167: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-20 15:49:49.547448: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-04-20 15:49:49.828909: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
# Dividir os dados de treinamento em treinamento e validação
# Embaralhando os dados para garantir que tanto a validação quanto o treinamento tenham todos os tipos de dados
ds_train = ds_train.shuffle(buffer_size=1000, seed=42)  

# Calcular o tamanho do conjunto de validação como 25% dos dados de treinamento
train_size = ds_train.cardinality().numpy()
validation_size = train_size // 4  # 25% dos dados de treinamento

# Dividindo os dados
ds_validation = ds_train.take(validation_size)
ds_train = ds_train.skip(validation_size)

# Verificar o número de exemplos em cada conjunto de dados
num_train_examples = ds_train.cardinality().numpy()
num_validation_examples = ds_validation.cardinality().numpy()
print("Número de exemplos de treinamento:", num_train_examples)
print("Número de exemplos de validação:", num_validation_examples)

Número de exemplos de treinamento: 40728
Número de exemplos de validação: 13575


In [6]:
df_validation = tfds.as_dataframe(ds_validation, info)
df_train = tfds.as_dataframe(ds_train, info)

: 